## Retriever And Chain With Langchain

In [26]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("uae_guide.pdf")
docs = loader.load()
docs

[Document(page_content="UNITED ARAB\nEMIRATES\nAn Expat's Handbook for Residing and Working in Abu Dhabi and DubaiMOVING TO THE ", metadata={'source': 'uae_guide.pdf', 'page': 0}),
 Document(page_content='CONTENTS\nDiscovering United Arab Emirates - A Sneak Peak Into\nWhat Awaits You\nResidential Leasing in the UAE\nAbu Dhabi\nDubai\nSelecting the Right School: A Guide for Parents and\nStudents\nApplying for a Residency Permit in the UAE\nOpening a Bank Account in the UAE\nObtaining a Driving License in the UAE\nCompensation Packages: Understanding Salary\nBenefits in the UAE\nLiving Expenses', metadata={'source': 'uae_guide.pdf', 'page': 1}),
 Document(page_content="UNITED ARAB\nEMIRATES\nA SNEAK PEAK INTO WHAT\nAWAITS YOU\nThe UAE, a favored expatriate destination, boasts a flourishing business sector and contemporary living standards. It is composed of seven\nemirates, which gained independence and came together as a country in 1971, following the end of British military protection 

In [27]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
text_splitter.split_documents(docs)[:5]

[Document(page_content="UNITED ARAB\nEMIRATES\nAn Expat's Handbook for Residing and Working in Abu Dhabi and DubaiMOVING TO THE", metadata={'source': 'uae_guide.pdf', 'page': 0}),
 Document(page_content='CONTENTS\nDiscovering United Arab Emirates - A Sneak Peak Into\nWhat Awaits You\nResidential Leasing in the UAE\nAbu Dhabi\nDubai\nSelecting the Right School: A Guide for Parents and\nStudents\nApplying for a Residency Permit in the UAE\nOpening a Bank Account in the UAE\nObtaining a Driving License in the UAE\nCompensation Packages: Understanding Salary\nBenefits in the UAE\nLiving Expenses', metadata={'source': 'uae_guide.pdf', 'page': 1}),
 Document(page_content="UNITED ARAB\nEMIRATES\nA SNEAK PEAK INTO WHAT\nAWAITS YOU\nThe UAE, a favored expatriate destination, boasts a flourishing business sector and contemporary living standards. It is composed of seven\nemirates, which gained independence and came together as a country in 1971, following the end of British military protection i

In [28]:
documents=text_splitter.split_documents(docs)
documents

[Document(page_content="UNITED ARAB\nEMIRATES\nAn Expat's Handbook for Residing and Working in Abu Dhabi and DubaiMOVING TO THE", metadata={'source': 'uae_guide.pdf', 'page': 0}),
 Document(page_content='CONTENTS\nDiscovering United Arab Emirates - A Sneak Peak Into\nWhat Awaits You\nResidential Leasing in the UAE\nAbu Dhabi\nDubai\nSelecting the Right School: A Guide for Parents and\nStudents\nApplying for a Residency Permit in the UAE\nOpening a Bank Account in the UAE\nObtaining a Driving License in the UAE\nCompensation Packages: Understanding Salary\nBenefits in the UAE\nLiving Expenses', metadata={'source': 'uae_guide.pdf', 'page': 1}),
 Document(page_content="UNITED ARAB\nEMIRATES\nA SNEAK PEAK INTO WHAT\nAWAITS YOU\nThe UAE, a favored expatriate destination, boasts a flourishing business sector and contemporary living standards. It is composed of seven\nemirates, which gained independence and came together as a country in 1971, following the end of British military protection i

In [30]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

#db=FAISS.from_documents(documents[:30],OllamaEmbeddings(model="gemma:2b"))
db=FAISS.from_documents(documents,OllamaEmbeddings(model="gemma:2b"))

In [42]:
db

In [31]:
query="Al Reem Island "
result=db.similarity_search(query)
result[0].page_content

'renowned for its famous tourist attractions, cafes, restaurants, malls, hotels, schools, and hospitals.\nHowever, the luxury amenities and tourist appeal make accommodation on Yas Island quite\nexpensive. The island is located 20 km East of Abu Dhabi city center and can be reached within\napproximately 15 minutes by car from Abu Dhabi airport.'

Use llm with prompt template

In [33]:
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
llm=Ollama(model="gemma:2b")
llm

Ollama(model='gemma:2b')

In [34]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

In [35]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [36]:
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000027B2FD63E50>)

In [37]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [43]:
response=retrieval_chain.invoke({"input":"private school"})


In [44]:
response['answer']

"**Step 1**: Based on the context, private schools can range from 15,000 Dirhams to 80,000 Dirhams per year.\n\n\n**Step 2**: The cost of private school tuition in Abu Dhabi and Dubai can vary depending on the school's curriculum, location, and ranking.\n\n\n**Step 3**: Factors to consider when selecting a private school include tuition fees, curriculum, location, ranking, and exam results.\n\n\n**Step 4**: The education sector in the country is overseen by the Ministry of Education. In Abu Dhabi, the Abu Dhabi Department of Education and Knowledge (ADEK) is dedicated to developing and monitoring the education system in the Emirate. Meanwhile, in Dubai, the Knowledge and Human Development Authority (KHDA) is responsible for inspecting all private schools to ensure the quality of education. Both organizations are also responsible for ranking schools.\n\n\n**Step 5**: To ensure a quality education, it is important to consider the school's ranking, curriculum, and facilities offered."